In [1]:
import pandas as pd

In [2]:
lga_codes = pd.read_csv('Data/lga_code_and_names.csv')
lga_codes.head()

,LGA_CODE,LGA_NAME
0,LGA10050,Albury
1,LGA10130,Armidale Regional
2,LGA10250,Ballina
3,LGA10300,Balranald
4,LGA10470,Bathurst Regional


In [3]:
def export_rent(inpath_rent, outname_rent, detail = 'overall'):
    rent = pd.read_excel(inpath_rent, sheet_name = 1)
    rent = rent.drop(rent.index[0:2])
    rent.columns = rent.iloc[0]
    rent = rent.drop(rent.index[0])
    rent = rent.drop(['Quarterly change in Median Weekly Rent', 'Annual change in Median Weekly Rent',
                          'Quarterly change in New Bonds Lodged', 'Annual change in New Bonds Lodged'], axis = 1)
    cols = ['gmr', 'gs', 'rings', 'lga', 'dwelling_type', 'bedroom_no', 'weekly_rent_q1', 'weekly_rent_median',
            'weekly_rent_q3', 'new_bonds', 'total_bonds']
    rent.columns = cols
    rent = rent[(rent.gmr == 'Total') & (rent.gs == 'Total') & (rent.rings == 'Total') & (rent.lga != 'Total')]
    rent = rent.replace('-', '')
    if detail == 'dwelling':
        rent_by_detail = rent[(rent.dwelling_type != 'Total') & (rent.bedroom_no == 'Total')]
        rent_by_detail = rent_by_detail[['lga', 'dwelling_type', 'weekly_rent_median']]
        rent_by_detail.weekly_rent_median = pd.to_numeric(rent_by_detail.weekly_rent_median)
        rent_by_detail = pd.pivot_table(rent_by_detail, index = 'lga', columns = 'dwelling_type',
                                        values = 'weekly_rent_median')
        rent_by_detail = rent_by_detail.reset_index()
        rent_by_detail.columns = ['LGA_NAME', 'flat', 'house', 'other', 'townhouse']
        rent_by_detail = rent_by_detail.fillna(0)
    elif detail == 'bedroom':
        rent_by_detail = rent[(rent.dwelling_type == 'Total') & (rent.bedroom_no != 'Total')]
        rent_by_detail = rent_by_detail[['lga', 'bedroom_no', 'weekly_rent_median']]
        rent_by_detail.weekly_rent_median = pd.to_numeric(rent_by_detail.weekly_rent_median)
        rent_by_detail = pd.pivot_table(rent_by_detail, index = 'lga', columns = 'bedroom_no',
                                        values = 'weekly_rent_median')
        rent_by_detail = rent_by_detail.reset_index()
        rent_by_detail.columns = ['LGA_NAME', 'one_br', 'two_br', 'three_br', 'four_or_more_br', 'bedsitter',
                                  'not_specified']
        rent_by_detail = rent_by_detail.fillna(0)
    else:
        rent_by_detail = rent[(rent.dwelling_type == 'Total') & (rent.bedroom_no == 'Total')]
        rent_by_detail = rent_by_detail[['lga', 'weekly_rent_median']]
        rent_by_detail.weekly_rent_median = pd.to_numeric(rent_by_detail.weekly_rent_median)
        rent_by_detail.columns = ['LGA_NAME', 'weekly_rent_median']
        rent_by_detail = rent_by_detail.fillna(0)
    rent_final = lga_codes.merge(rent_by_detail, on = 'LGA_NAME', how = 'inner')
    rent_final.to_csv(outname_rent, index = False)

In [4]:
export_rent('Data/house_rent_sales/Rent-Tables-Jun-Quarter-2019.xlsx',
            'Data/house_rent_sales/house_rent_0619_overall.csv')
export_rent('Data/house_rent_sales/Rent-Tables-Jun-Quarter-2019.xlsx',
            'Data/house_rent_sales/house_rent_0619_dwelling.csv', detail = 'dwelling')
export_rent('Data/house_rent_sales/Rent-Tables-Jun-Quarter-2019.xlsx',
            'Data/house_rent_sales/house_rent_0619_bedroom.csv', detail = 'bedroom')

In [5]:
export_rent('Data/house_rent_sales/Rent-Tables-Sep-Quarter-2017.xlsx',
            'Data/house_rent_sales/house_rent_0917_overall.csv')
export_rent('Data/house_rent_sales/Rent-Tables-Sep-Quarter-2017.xlsx',
            'Data/house_rent_sales/house_rent_0917_dwelling.csv', detail = 'dwelling')
export_rent('Data/house_rent_sales/Rent-Tables-Sep-Quarter-2017.xlsx',
            'Data/house_rent_sales/house_rent_0917_bedroom.csv', detail = 'bedroom')

In [6]:
def export_sales(inpath_sales, outname_sales, stat = 'mean', house_type = False):
    sales = pd.read_excel(inpath_sales, sheet_name = 1)
    sales = sales.drop(sales.index[0:3])
    sales.columns = sales.iloc[0]
    sales = sales.drop(sales.index[0])
    sales = sales.drop(['Qtly change in Median', 'Annual change in Median', 'Qtly change in Count',
                        'Annual change in Count'], axis = 1)
    cols = ['gmr', 'gs', 'rings', 'lga', 'dwelling_type', 'sales_q1', 'median', 'sales_q3', 'mean',
            'sales_count']
    sales.columns = cols
    sales = sales[(sales.gmr == 'Total') & (sales.gs == 'Total') & (sales.rings == 'Total') & (sales.lga != 'Total')]
    sales = sales[['lga', 'dwelling_type', stat]]
    if house_type:
        sales_by_detail = sales[(sales.dwelling_type != 'Total')]
        sales_by_detail[stat] = sales_by_detail[stat].replace('-', 0)
        sales_by_detail[stat] = pd.to_numeric(sales_by_detail[stat])
        sales_by_detail = pd.pivot_table(sales_by_detail, index = 'lga', columns = 'dwelling_type', values = stat)
        sales_by_detail = sales_by_detail.reset_index()
        sales_by_detail.columns = ['lga', 'nonstrata', 'strata']
        sales_by_detail = sales_by_detail.fillna(0)
        sales_by_detail.columns = ['LGA_NAME', stat + '_nonstrata', stat + '_strata']
    else:
        sales_by_detail = sales[(sales.dwelling_type == 'Total')]
        sales_by_detail = sales_by_detail[['lga', stat]]
        sales_by_detail[stat] = sales[stat].replace('-', 0)
        sales_by_detail[stat] = pd.to_numeric(sales_by_detail[stat])
        sales_by_detail = sales_by_detail.fillna(0)
        sales_by_detail.columns = ['LGA_NAME', stat + '_sales']
    sales_final = lga_codes.merge(sales_by_detail, on = 'LGA_NAME', how = 'inner')
    sales_final.to_csv(outname_sales, index = False)

In [6]:
export_sales('Data/house_rent_sales/Sales-Tables-Mar-Quarter-2019.xlsx',
             'Data/house_rent_sales/house_sales_0319_overall_mean.csv')
export_sales('Data/house_rent_sales/Sales-Tables-Mar-Quarter-2019.xlsx',
             'Data/house_rent_sales/house_sales_0319_overall_median.csv', stat = 'median')
export_sales('Data/house_rent_sales/Sales-Tables-Mar-Quarter-2019.xlsx',
             'Data/house_rent_sales/house_sales_0319_dwelling_mean.csv', house_type = True)
export_sales('Data/house_rent_sales/Sales-Tables-Mar-Quarter-2019.xlsx',
             'Data/house_rent_sales/house_sales_0319_dwelling_median.csv', stat = 'median', house_type = True)

/Users/jvlpz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [7]:
export_sales('Data/house_rent_sales/Sales-Tables-Jun-Quarter-2017.xlsx',
             'Data/house_rent_sales/house_sales_0617_overall_mean.csv')
export_sales('Data/house_rent_sales/Sales-Tables-Jun-Quarter-2017.xlsx',
             'Data/house_rent_sales/house_sales_0617_overall_median.csv', stat = 'median')
export_sales('Data/house_rent_sales/Sales-Tables-Jun-Quarter-2017.xlsx',
             'Data/house_rent_sales/house_sales_0617_dwelling_mean.csv', house_type = True)
export_sales('Data/house_rent_sales/Sales-Tables-Jun-Quarter-2017.xlsx',
             'Data/house_rent_sales/house_sales_0617_dwelling_median.csv', stat = 'median', house_type = True)

/Users/jvlpz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/jvlpz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
